In [86]:
import os
from openai import OpenAI
from datetime import datetime
import os
import sys
import json
import base64
import requests
from PIL import Image
import matplotlib.pyplot as plt
from prettyprinter import pprint
from pydantic import BaseModel

from utils_response import *

In [81]:
file_path = os.path.expanduser('~/Desktop/plants2')
jpg_files = [file for file in os.listdir(file_path) if file.endswith('.jpg')]

filename = jpg_files[0]

In [82]:
def encode_image(file_path, filename):
    image_path = os.path.join(file_path, filename)
    with open(image_path, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')
    
def display_image(file_path, filename):
    image_path = os.path.join(file_path, filename)
    img = Image.open(image_path)

    fig, axes = plt.subplots(1,1, figsize=(4,4))
    axes.imshow(img)
    axes.axis('off')
    plt.show()

base64_image = encode_image(file_path, filename)
# display_image(file_path, filename)

In [83]:
class PlantResponse(BaseModel):
    name: str
    confidence: str
    desc: str
        

client = OpenAI()

system_message_text = ('Respond in json format with name: the most common plant name, use _ if the name uses two words; ' +
                       'confidence: low/medium or high depending on your confidence; ' +
                       'desc: short description ')
prompt_message_text = "What is the plant in this image?"
image_url = f"data:image/jpeg;base64,{base64_image}"
detail = 'high'

messages = []
message_sys = {'role': 'system', 'content': [
    {
        'type': 'text',
        'text': system_message_text
    }
]}

message_prompt = {"role": "user","content": [
    {
        "type": "text",
        "text": "What is the plant in this image?"
    },
    {
        "type": "image_url",
        "image_url": {
            "url": image_url,
            "detail": detail
          },
        }
    ],
}
messages.append(message_sys)
messages.append(message_prompt)


response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
  messages=messages,
  response_format=PlantResponse,#{'type': 'json_object'},
  max_tokens=300,
)

print(response.choices[0].message.content)

{"name":"Skunk_Cabbage","desc":"A plant with large green leaves, often found in wetland areas, known for its skunky odor."}


In [59]:
print(get_response_cost(response))
get_response_model(response)
a = get_response_content(response)

print(a)

0.0024124999999999997
{"name":"Helenium","desc":"A perennial plant known for its vibrant, daisy-like flowers with red and yellow hues."}


In [84]:
flowers = []
count = 0
for filename in jpg_files:
    base64_image = encode_image(file_path, filename)

    system_message_text = 'Respond in json format with name: the most common plant name, use _ if the name uses two words, desc: short description'
    prompt_message_text = "What is the plant in this image?"
    image_url = f"data:image/jpeg;base64,{base64_image}"
    detail = 'high'

    messages = []
    message_sys = {'role': 'system', 'content': [
        {
            'type': 'text',
            'text': system_message_text
        }
    ]}

    message_prompt = {"role": "user","content": [
        {
            "type": "text",
            "text": "What is the plant in this image?"
        },
        {
            "type": "image_url",
            "image_url": {
                "url": image_url,
                "detail": detail
            },
            }
        ],
    }
    messages.append(message_sys)
    messages.append(message_prompt)
    response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=messages,
        response_format=PlantResponse,#{'type': 'json_object'},
        max_tokens=300
        )
    response_dict = json.loads(response.choices[0].message.content)
    flowers.append((filename, response_dict['name']))
    count += 1

    if count == 100:
        break
    

('skunk_cabbage.jpg', 'Skunk_Cabbage')
('PXL_20240907_104930484.jpg', 'Salvia')
('silver_ragwort.jpg', 'Dusty_Miller')
('PXL_20240907_105952127.MP.jpg', 'Cyclamen')
('dahlia.jpg', 'Dahlia')
('PXL_20240907_115158758.jpg', 'Helenium')
('chrysanthemum.jpg', 'Dahlia')
('japanese_maple.jpg', 'Japanese_maple')
('PXL_20240907_103100075.jpg', 'Coreopsis')
('PXL_20240907_115027852.jpg', 'Wisteria')
('euphorbia.jpg', 'Euphorbia')
('PXL_20240907_113623641.jpg', 'Heather')
('PXL_20240907_115141514.jpg', 'Black_Eyed_Susan')
('Helenium.jpg', 'Helenium')
('PXL_20240907_110128752.jpg', 'Hare_barley')
('PXL_20240907_104937426.jpg', 'blue_sage')
('PXL_20240907_114408420.jpg', 'Snapdragon')
('siberian_bugloss.jpg', 'Brunnera_macrophylla')
('ageratum.jpg', 'Flossflower')
('PXL_20240907_115025424.jpg', 'Wisteria')
('PXL_20240907_104552053.jpg', 'Magnolia')
[('skunk_cabbage.jpg', 'Skunk_Cabbage'), ('PXL_20240907_104930484.jpg', 'Salvia'), ('silver_ragwort.jpg', 'Dusty_Miller'), ('PXL_20240907_105952127.MP.j

In [72]:
# ??get_response_content
response.choices[0].message.content
a = json.loads(response.choices[0].messagea.content)

In [74]:
a['name']

'Helenium'

In [85]:
file_path = os.path.expanduser('~/Desktop/plants2')

# Function to handle renaming
def rename_files(flowers, file_path):
    # Dictionary to track name counts
    name_counts = {}

    for filename, name in flowers:
        filename = filename.lower()
        name = name.lower()
        # Construct the new filename with a count if necessary
        if name in name_counts:
            name_counts[name] += 1
            new_name = f"{name}_{name_counts[name]}.jpg"
        else:
            name_counts[name] = 1
            new_name = f"{name}.jpg"
        
        # Full paths for the renaming operation
        old_file_path = os.path.join(file_path, filename)
        new_file_path = os.path.join(file_path, new_name)
        
        # Rename the file
        if os.path.exists(old_file_path):
            os.rename(old_file_path, new_file_path)
            print(f"Renamed: {old_file_path} -> {new_file_path}")
        else:
            print(f"File not found: {old_file_path}")
        
# Now call the function with your flowers list and file path
rename_files(flowers, file_path)

Renamed: /Users/jamesbyers/Desktop/plants2/skunk_cabbage.jpg -> /Users/jamesbyers/Desktop/plants2/skunk_cabbage.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/pxl_20240907_104930484.jpg -> /Users/jamesbyers/Desktop/plants2/salvia.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/silver_ragwort.jpg -> /Users/jamesbyers/Desktop/plants2/dusty_miller.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/pxl_20240907_105952127.mp.jpg -> /Users/jamesbyers/Desktop/plants2/cyclamen.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/dahlia.jpg -> /Users/jamesbyers/Desktop/plants2/dahlia.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/pxl_20240907_115158758.jpg -> /Users/jamesbyers/Desktop/plants2/helenium.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/chrysanthemum.jpg -> /Users/jamesbyers/Desktop/plants2/dahlia_2.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/japanese_maple.jpg -> /Users/jamesbyers/Desktop/plants2/japanese_maple.jpg
Renamed: /Users/jamesbyers/Desktop/plants2/pxl_20240907_103100075.jpg -> /Us